In [1]:

import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split



In [2]:
import torch
import numpy as np

system = "cy"
x_train = np.load(f'/data5/chengjingwen/data/{system}/uv.npy')
x_train = torch.tensor(x_train, dtype=torch.float32)  
print(x_train.shape)
num_trajectories, total_steps, num_var, x_dim, y_dim = x_train.shape

xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  
xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  


data = (x_train - xmin) / (xmax - xmin)


print(f"Training data normalized: min={data.min().item()}, max={data.max().item()}")


# torch.save({'xmin': xmin, 'xmax': xmax}, f'/data4/chengjingwen/kdd25-main 3/data/lo/normalization_params.pth')

num_train = int(num_trajectories * 0.8)  # 80%的轨迹数
random_indices = torch.randperm(num_trajectories)[:num_train]  
data = data[random_indices]
num_trajectories, total_steps, num_var, x_dim, y_dim = data.shape


torch.Size([50, 200, 2, 128, 64])
Training data normalized: min=0.0, max=1.0


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class MultiStepInputDataset(Dataset):
    def __init__(self, data, history_steps=10):
        """
        data: Tensor, shape (num_trajectories, num_steps, num_variables, x_dim, y_dim)
        history_steps: how many past steps to use as input
        """
        self.data = data
        self.history_steps = history_steps
        self.num_trajectories, self.num_steps, self.num_variables, self.x_dim, self.y_dim = data.shape

        # 确保有足够时间步能取 history_steps + 1（因为input要10步，output是第11步）
        if self.num_steps <= self.history_steps:
            raise ValueError("num_steps must be greater than history_steps")

    def __len__(self):
        # 每条轨迹生成 (num_steps - history_steps) 个样本
        return self.num_trajectories * (self.num_steps - self.history_steps)

    def __getitem__(self, idx):
        # 确定轨迹编号和在轨迹内的起点时间步
        trajectory_idx = idx // (self.num_steps - self.history_steps)
        time_idx = idx % (self.num_steps - self.history_steps)

        # input: 连续 history_steps 个时间步，堆叠成channel
        input_data = self.data[trajectory_idx, time_idx : time_idx + self.history_steps]  # (history_steps, num_variables, x_dim, y_dim)
        

        # target: 预测的第 history_steps 后的一个时间步
        target_data = self.data[trajectory_idx, time_idx + self.history_steps].unsqueeze(0)  # (1, num_variables, x_dim, y_dim)

        return input_data, target_data


# train_data = data

# train_dataset = MultiStepInputDataset(train_data, history_steps=10)

# # 随机划分 80% train, 20% val
# train_size = int(0.8 * len(train_dataset))
# val_size = len(train_dataset) - train_size
# train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # 打印一个batch看看
# for inputs, targets in train_loader:
#     print(f"Input shape: {inputs.shape}, Target shape: {targets.shape}")
#     num_variables = inputs.shape[1]
#     break  # 只打印一次

In [2]:
from convlstm import ConvLSTM
import torch
import torch.nn as nn
class my_ConvLSTM(nn.Module):
    """

    inputs: [B, time_step, num_var, x_dim, y_dim] or [B, num_var, x_dim, y_dim]
    outputs: [B, pred_steps, num_var, x_dim, y_dim]
    """
    def __init__(self, num_var, x_dim=128, y_dim=128, hidden_dim=128, num_layers=3, n=1):
        super(my_ConvLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.n = n  # Default prediction steps
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.num_var = num_var
        self.cell = ConvLSTM(
            input_dim = num_var, 
            hidden_dim = hidden_dim,
            kernel_size = (3,3),
            num_layers = 3,    # make sure len(hidden_dim) = num_layers
            batch_first=True, 
            bias=True, 
            return_all_layers=False)

        self.final_conv = nn.Conv2d(in_channels=hidden_dim, out_channels=num_var, kernel_size=3, padding = 1)

    def forward(self, x, n=None):
        if n is None:  # Use default n if not provided
            n = self.n
            
        if len(x.shape) == 4:   # add time dimension
            x = x.unsqueeze(1)
            
        # Pass through LSTM
        layer_output_list, last_state_list = self.cell(x)   # x: [batch_size, T, num_var, input_dim, input_dim]
        (h,c) = last_state_list[0]
        last_state_list = [(h, c)] * self.num_layers
        
        output = layer_output_list[0][:,-1,:,:]  # 最后一层layer、最后一个时间步的输出  output:[batch_size, hidden_dim, input_dim, input_dim]
        final_output = self.final_conv(output)  # [batch_size, num_var, input_dim, input_dim]
        y = [final_output]
        
        # Multi-step prediction
        for _ in range(1, n):
            last_output = y[-1].unsqueeze(1)  # [batch_size, 1, hidden_dim, input_dim, input_dim]
            
            layer_output_list, last_state_list = self.cell(last_output, last_state_list)   
            (h,c) = last_state_list[0]
            last_state_list = [(h, c)] * self.num_layers
            
            output = layer_output_list[0][:,-1,:,:]  #[batch_size, hidden_dim, input_dim, input_dim]
            final_output = self.final_conv(output)  # [batch_size, num_var, input_dim, input_dim]
            
            y.append(final_output)
        final_outputs = torch.stack(y, dim=1).reshape(-1, self.num_var, self.x_dim, self.y_dim)
        # y_tensor: [batch_size, n, hidden_dim, input_dim, input_dim]
       
        # Return last prediction and all predictions
        return final_outputs.reshape(-1, n, self.num_var, self.x_dim, self.y_dim)  # [batch_size, n, num_var, input_dim, input_dim]

In [5]:
# test
x = torch.rand((32, 10, 2, 128, 128))
convlstm = my_ConvLSTM(2, 128, 256)   #
final_output = convlstm(x)
num_params = sum(p.numel() for p in convlstm.parameters())
print(f"模型总参数量: {num_params / 1e6:.2f} M")

模型总参数量: 2.96 M


In [9]:
print(final_output.shape)

torch.Size([32, 1, 2, 128, 128])


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm

x_dim = 128
y_dim = 128
input_step = 10
pred_step = 1
num_epochs = 30
batch_size = 8
device = "cuda:7" 
lr_min = 1e-5
hidden_dim = 64
num_layers = 3
train_data = data

train_dataset = MultiStepInputDataset(train_data, history_steps=10)

# 随机划分 80% train, 20% val
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 打印一个batch看看
for inputs, targets in train_loader:
    print(f"Input shape: {inputs.shape}, Target shape: {targets.shape}")
    num_variables = inputs.shape[1]
    break  # 只打印一次


Conv_LSTM = my_ConvLSTM(num_var, x_dim=x_dim, y_dim = y_dim, hidden_dim=hidden_dim, num_layers=num_layers, n=pred_step).to(device)


optimizer = Adam(list(Conv_LSTM.parameters()), lr=1e-3)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True
)


criterion = nn.MSELoss()


for epoch in range(num_epochs):
    Conv_LSTM.train()
   
    total_loss = 0.0
    
    for batch_idx, (input_data, target_data) in tqdm(enumerate(train_loader), total = len(train_loader)):
        
        input_data = input_data.to(device)  # Shape: [batch_size, num_variables, x_dim, y_dim]
        target_data = target_data.to(device)  # Shape: [batch_size, pred_step, num_variables, x_dim, y_dim]
        
        
        pred_data = Conv_LSTM(input_data)
        
        
        loss = criterion(pred_data, target_data)
        total_loss += loss.item()

        
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
    avg_loss = total_loss / len(train_loader)
    scheduler.step(avg_loss)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Reconstruction Loss: {avg_loss:.2e}")
    print(f"learning rate for lstm:{optimizer.param_groups[0]['lr']}")

    if optimizer.param_groups[0]['lr'] < lr_min:
        print(f"Learning rate below threshold ({lr_min}). Stopping early.")
        break    
       

    

print("Training complete!")


#### 为啥这么慢！！！！

Input shape: torch.Size([8, 10, 3, 128, 128]), Target shape: torch.Size([8, 1, 3, 128, 128])


/data3/chengjingwen/.conda/envs/lmenv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
100%|██████████| 720/720 [02:15<00:00,  5.32it/s]


Epoch [1/100], Reconstruction Loss: 1.30e-02
learning rate for lstm:0.001


100%|██████████| 720/720 [01:56<00:00,  6.19it/s]


Epoch [2/100], Reconstruction Loss: 3.65e-04
learning rate for lstm:0.001


...

Epoch [16/100], Reconstruction Loss: 3.94e-05
learning rate for lstm:0.001


100%|██████████| 720/720 [01:56<00:00,  6.19it/s]


Epoch [17/100], Reconstruction Loss: 3.48e-05
learning rate for lstm:0.001


 54%|█████▍    | 388/720 [01:02<00:53,  6.18it/s]


KeyboardInterrupt: 

In [17]:
# num_epochs = 60

# optimizer = Adam(list(Conv_LSTM.parameters()), lr=1e-4)

# for epoch in range(num_epochs):
#     Conv_LSTM.train()
   
#     total_loss = 0.0
    
#     for batch_idx, (input_data, target_data) in enumerate(train_loader):
#        
#         input_data = input_data.to(device)  # Shape: [batch_size, num_variables, x_dim, y_dim]
#         target_data = target_data.to(device)  # Shape: [batch_size, pred_step, num_variables, x_dim, y_dim]
        
        
#         pred_data = Conv_LSTM(input_data)
        
#         
#         loss = criterion(pred_data, target_data)
#         total_loss += loss.item()

#         
#         optimizer.zero_grad()
        
#         loss.backward()
#         optimizer.step()
        
#     avg_loss = total_loss / len(train_loader)
#     scheduler.step(avg_loss)
#     print(f"Epoch [{epoch + 1}/{num_epochs}], Reconstruction Loss: {avg_loss:.2e}")
#     print(f"learning rate for lstm:{optimizer.param_groups[0]['lr']}")

#     if optimizer.param_groups[0]['lr'] < lr_min:
#         print(f"Learning rate below threshold ({lr_min}). Stopping early.")
#         break    
       

    

# print("Training complete!")

Epoch [1/60], Reconstruction Loss: 1.60e-03
learning rate for lstm:0.0001
Epoch [2/60], Reconstruction Loss: 1.57e-03
learning rate for lstm:0.0001
Epoch [3/60], Reconstruction Loss: 1.57e-03
learning rate for lstm:0.0001
Epoch [4/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [5/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [6/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [7/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [8/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [9/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [10/60], Reconstruction Loss: 1.55e-03
learning rate for lstm:0.0001
Epoch [11/60], Reconstruction Loss: 1.56e-03
learning rate for lstm:0.0001
Epoch [12/60], Reconstruction Loss: 1.55e-03
learning rate for lstm:0.0001
Epoch [13/60], Reconstruction Loss: 1.55e-03
learning rate for lstm:0.0001
Epoch [14/60], Reconstruction Loss

In [6]:

# torch.save(Conv_LSTM.state_dict(), f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/{system}/convlstm_{hidden_dim}_{num_layers}.pth")

# print("Models saved successfully!")

Models saved successfully!


In [3]:
import torch
import numpy as np

system = "sevir_tem"
x_test = np.load(f'/data5/chengjingwen/{system}/uv_test.npy')
x_test = torch.tensor(x_test, dtype=torch.float32)  # 转换为 Tensor

x_train = np.load(f'/data5/chengjingwen/{system}/uv.npy')
x_train = torch.tensor(x_train, dtype=torch.float32)  

num_trajectories, total_steps, num_var, x_dim, y_dim = x_test.shape

xmin = x_train.amin(dim=(0, 1, 3, 4), keepdim=True)  
xmax = x_train.amax(dim=(0, 1, 3, 4), keepdim=True)  
del x_train

x_test_normalized = (x_test - xmin) / (xmax - xmin)


print(f"Test data normalized: min={x_test_normalized.min().item()}, max={x_test_normalized.max().item()}")

Test data normalized: min=0.0, max=0.913608968257904


In [4]:
import numpy as np
import torch
from torchmetrics.functional import structural_similarity_index_measure as ssim

import matplotlib.pyplot as plt
from IPython.display import clear_output
pred_step = 1  
pred_steps = 30
start_time = 10

def compute_nmse(predictions, targets):
    """计算 NMSE"""
    return torch.mean(((predictions - targets) ** 2)) / torch.mean((targets ** 2))

def compute_rmse(predictions, targets):
    """计算 RMSE"""
    mse = torch.mean((predictions - targets) ** 2)
    return torch.sqrt(mse)


def test_model(convlstm, test_data, device):
    convlstm.eval()

    num_trajectories, T, num_variables, x_dim, y_dim = test_data.shape

    nmse_list = []
    ssim_list = []
    rmse_list = []
    
    with torch.no_grad():
        for trajectory_idx in range(num_trajectories):
            print(f"now for {trajectory_idx}")
            trajectory = test_data[trajectory_idx]  # shape [100, num_variables, x_dim, y_dim]
            
            targets = trajectory[start_time :start_time + pred_steps].to(device)  

        
            predictions = torch.zeros((pred_steps, num_variables, x_dim, y_dim)).to(device)

            for t in range(0,pred_steps,pred_step):  
                if(t==0):
                    input_data = trajectory[start_time-10:start_time].unsqueeze(0).to(device)  # shape: [1, 10, num_variables, x_dim, y_dim]
                # else: 
                #     input_data = preds[-1, :, :, :].unsqueeze(0)
                    
                preds = Conv_LSTM(input_data)  # shape: [1, pred_step, num_variables, x_dim, y_dim]
                input_data = input_data[:,1:]
                input_data = torch.cat([input_data, preds], dim = 1)
                
                predictions[t : t + pred_step] = preds
                
                # ipdb.set_trace()
            # if trajectory_idx == 0:
            #     for t in range(50):
            #         plt.figure(figsize=(12, 4))

            #         # Ground Truth
            #         plt.subplot(131)
            #         plt.title(f"Ground Truth (step={ t + 1})")
            #         plt.imshow(targets[t, 0].detach().cpu().numpy(), cmap="jet", vmin=-1, vmax=1)  
            #         plt.colorbar()

            #         # Prediction
            #         plt.subplot(132)
            #         plt.title("Prediction")
            #         plt.imshow(predictions[t, 0].detach().cpu().numpy(), cmap="jet", vmin=-1, vmax=1)  
            #         plt.colorbar()

            #         # Difference and MSE
            #         mse = torch.mean((targets[t, 0] - predictions[t, 0]) ** 2).item()
            #         plt.subplot(133)
            #         plt.title(f"Difference (MSE={mse:.5f})")
            #         plt.imshow(torch.abs(targets[t, 0] - predictions[t, 0]).detach().cpu().numpy(), cmap="jet")
            #         plt.colorbar()

            #         plt.show()
            #         clear_output(wait=True)  
                

        
            ssim_values = ssim(predictions, targets, data_range=1.0).item() 
            nmse_trajectory = compute_nmse(predictions, targets).item() 
            rmse_trajectory = compute_rmse(predictions, targets).item()  

            nmse_list.append(nmse_trajectory)
            ssim_list.append(ssim_values)
            rmse_list.append(rmse_trajectory)

    
        nmse_mean, nmse_std = np.mean(nmse_list), np.std(nmse_list)
        ssim_mean, ssim_std = np.mean(ssim_list), np.std(ssim_list)
        rmse_mean, rmse_std = np.mean(rmse_list), np.std(rmse_list)

        return {
            "NMSE_mean": nmse_mean,
            "NMSE_std": nmse_std,
            "SSIM_mean": ssim_mean,
            "SSIM_std": ssim_std,
            "RMSE_mean": rmse_mean,
            "RMSE_std": rmse_std
        }

with torch.no_grad():
    device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

    hidden_dim = 128
    num_layers = 3
    x_dim = 128
    y_dim = 128
    Conv_LSTM = my_ConvLSTM(num_var, x_dim=x_dim, y_dim = y_dim, hidden_dim=hidden_dim, num_layers=3, n=pred_step).to(device)
    Conv_LSTM.load_state_dict(torch.load(f"/data3/chengjingwen/diffusion-mdpnet/trained_model/baseline/{system}/convlstm_{hidden_dim}_{num_layers}.pth"))
    Conv_LSTM.eval()

    print("Models loaded successfully!")


    test_data = x_test_normalized.to(device)  
    print(test_data.shape)

    results = test_model(Conv_LSTM, test_data, device)


    print(f"NMSE Mean: {results['NMSE_mean']}")
    print(f"NMSE Std: {results['NMSE_std']}")
    print(f"SSIM Mean: {results['SSIM_mean']}")
    print(f"SSIM Std: {results['SSIM_std']}")
    print(f"RMSE Mean: {results['RMSE_mean']}")
    print(f"RMSE Std: {results['RMSE_std']}")



/tmp/ipykernel_3251899/1648864482.py:111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Conv_LSTM.load_state_dict(torch.load(f"/data3/chengjingwen/diffusion-mdpnet/trained_m

Models loaded successfully!
torch.Size([50, 49, 1, 128, 128])
now for 0


/data3/chengjingwen/.conda/envs/lmenv/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


now for 1
now for 2
now for 3
now for 4
now for 5
now for 6
now for 7
now for 8
now for 9
now for 10
now for 11
now for 12
now for 13
now for 14
now for 15
now for 16
now for 17
now for 18
now for 19
now for 20
now for 21
now for 22
now for 23
now for 24
now for 25
now for 26
now for 27
now for 28
now for 29
now for 30
now for 31
now for 32
now for 33
now for 34
now for 35
now for 36
now for 37
now for 38
now for 39
now for 40
now for 41
now for 42
now for 43
now for 44
now for 45
now for 46
now for 47
now for 48
now for 49
NMSE Mean: 0.07461631279438734
NMSE Std: 0.03550012458813356
SSIM Mean: 0.7287774604558944
SSIM Std: 0.1195255062482466
RMSE Mean: 0.13682179391384125
RMSE Std: 0.02776724391270291


In [28]:
model = Conv_LSTM.to("cuda:3")
num_params = sum(p.numel() for p in model.parameters())
print(f"模型总参数量: {num_params / 1e6:.2f} M")

模型总参数量: 0.74 M


In [ ]:
print(test_data.shape)

torch.Size([40, 50, 3, 64, 64])
